# Prototyping an automatic well tie

## Outline

1. Building a Synthetic Well Log
1. Generating Synthetic Seismic trace
1. Generating Synthetic using portion of Synthetic Well
1. Dynamic Time Warping (DTW) for automatic well ties

This notebook will explore building a synthetic well log of alternating lithology units, namely shale and sand.

The synthetic well log will be convolved with a wavetlet and then will have random noise added.  This will be our synthetic seismic trace.

The we will then use only a portion of the synthetic well to simulate the portion recorded by wireline logging.  This will be used to create a synthetic for seismic-to-well tie purposes.

We will then explore using dtw-python to create an automatic well tie.

A portion of code from a 2017 SEG Tutorial on [Seismic Rock Physics](https://github.com/seg/tutorials-2017/blob/master/1706_Seismic_rock_physics/seismic_rock_physics.ipynb) will help guide some of the work to build a synthetic well.

## Library Imports

In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from bruges.filters import ricker
from bruges.reflection import shuey2